In [78]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
  llm=llm, 
  max_token_limit=120,
  return_messages=True,
)

prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful AI talking to a human"),
  MessagesPlaceholder(variable_name="history"),
  ("human", "{question}")
])

def load_memory(input):
  print(input)
  return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def ask(question):
  result = chain.invoke({
    "question": question,
  })
  memory.save_context({"input": question}, {"output": result.content})
  print(result)



In [79]:
ask("My naem is Yuls.")
ask("I live in Seoul.")
ask("What is my name?")

{'question': 'My naem is Yuls.'}
content='Hello Yuls! How can I assist you today?'
{'question': 'I live in Seoul.'}
content="That's great! Seoul is a vibrant and bustling city. How can I help you with anything related to Seoul or any other topic?"
{'question': 'What is my name?'}
content='Your name is Yuls, as you mentioned earlier. Is there anything specific you would like to know or discuss?'
